# Celeribity Recognition using Amazon Rekognition

***
This notebook provides a walkthrough of [celebrity recognition API](https://docs.aws.amazon.com/rekognition/latest/dg/celebrities.html) in Amazon Rekognition. You can quickly identify well known people in your video and image libraries to catalog footage and photos for marketing, advertising, and media industry use cases.
***

**Prerequisites:**

The user or role that executes the commands must have permissions in AWS Identity and Access Management (IAM) to perform those actions. AWS provides a set of managed policies that help you get started quickly. For our example, you need to apply the following minimum managed policies to your user or role:

* AmazonRekognitionFullAccess
* AmazonS3FullAccess

Be aware that we recommend you follow AWS IAM best practices for production implementations, which is out of scope fof this workshop.
***


## Initialize stuff

In [ ]:
# Initialise Notebook
import boto3
from IPython.display import HTML, display
from PIL import Image, ImageDraw, ImageFont
import time

In [ ]:
# Init clients
rekognition = boto3.client('rekognition')
s3 = boto3.client('s3')

In [ ]:
# S3 bucket that contains sample images and videos
bucketName = "ki-reinvent-content"

In [ ]:
# Create temporary directory
# This directory is not needed to call Rekognition APIs.
# We will only use this directory to download images from S3 bucket and drwaw bounding boxes
# around recognized celebrities to show them here in the notebook.

!mkdir m1tmp
tempFolder = 'm1tmp/'

## Recognize celebrities in image
***

In [ ]:
imageName = "friends.jpg"

In [ ]:
# Call Amazon Rekognition to recognize celebrities in the image
# https://docs.aws.amazon.com/rekognition/latest/dg/API_RecognizeCelebrities.html

recognizeCelebritiesResponse = rekognition.recognize_celebrities(
    Image={
        'S3Object': {
            'Bucket': bucketName,
            'Name': imageName,
        }
    }
)

In [ ]:
# Show JSON response returned by Rekognition Celebrity Recognition API
# In the JSON response below, you will see CelebrityFaces which contains information about recognized celebrities.
# For each recognized celebrity, you will see information like Name, Id, Urls and additional information about 
# their facial attributes.

display(recognizeCelebritiesResponse)

In [ ]:
# Define a function that will display image with bounded boxes around recognized celebrites
# We will call this function in next step
  
def drawBoundingBoxes (sourceImage, boxes):
    # blue, green, red, grey
    colors = ((220,220,220),(242,168,73),(76,182,252),(52,194,123))
    
    # Download image locally
    imageLocation = tempFolder+sourceImage
    s3.download_file(bucketName, sourceImage, imageLocation)

    # Draws BB on Image
    bbImage = Image.open(imageLocation)
    draw = ImageDraw.Draw(bbImage)
    width, height = bbImage.size
    col = 0
    maxcol = len(colors)
    line= 3
    for box in boxes:
        x1 = int(box[1]['Left'] * width)
        y1 = int(box[1]['Top'] * height)
        x2 = int(box[1]['Left'] * width + box[1]['Width'] * width)
        y2 = int(box[1]['Top'] * height + box[1]['Height']  * height)
        
        draw.text((x1,y1),box[0],colors[col])
        for l in range(line):
            draw.rectangle((x1-l,y1-l,x2+l,y2+l),outline=colors[col])
        col = (col+1)%maxcol
    
    imageFormat = "PNG"
    ext = sourceImage.lower()
    if(ext.endswith('jpg') or ext.endswith('jpeg')):
       imageFormat = 'JPEG'

    bbImage.save(imageLocation,format=imageFormat)

    display(bbImage)

In [ ]:
# Extract bounding box information from JSON response above and display image with bounding boxes around celebrites.

boxes = []
celebrities = recognizeCelebritiesResponse['CelebrityFaces']
for celebrity in celebrities:
    boxes.append ((celebrity['Name'], celebrity['Face']['BoundingBox']))
    
drawBoundingBoxes(imageName, boxes)

## Recognize celebrities in video
 Celebrity recognition in video is an async operation. 
 https://docs.aws.amazon.com/rekognition/latest/dg/API_StartCelebrityRecognition.html
 We first start celebrity recognition job which returns a Job Id.
 We can then call get_celebrity_recognition to get the job status and after job is complete, we can get 
 celebrity metadata.
 In production use cases, you would usually use StepFucntion or SNS topic to get notified when job is complete.
***

In [ ]:
videoName = "serverless-bytes.mov"

In [ ]:
# Start celebrity recognition job
startCelebrityRekognition = rekognition.start_celebrity_recognition(
    Video={
        'S3Object': {
            'Bucket': bucketName,
            'Name': videoName,
        }
    },
)

celebrityJobId = startCelebrityRekognition['JobId']
display("Job Id: {0}".format(celebrityJobId))

In [ ]:
# Wait for celebrity recognition job to complete
# In production use cases, you would usually use StepFucntion or SNS topic to get notified when job is complete.
getCelebrityRecognition = rekognition.get_celebrity_recognition(
    JobId=celebrityJobId,
    SortBy='TIMESTAMP'
)

while(getCelebrityRecognition['JobStatus'] == 'IN_PROGRESS'):
    time.sleep(5)
    print('.', end='')
 
    getCelebrityRecognition = rekognition.get_celebrity_recognition(
    JobId=celebrityJobId,
    SortBy='TIMESTAMP')
    
display(getCelebrityRecognition['JobStatus'])

In [ ]:
# Show JSON response returned by Rekognition Celebrity Recognition API
# In the JSON response below, you will see list Celebrities which contains information about recognized celebrities.
# For each recognized celebrity, you will see information like Timestamp, Name, Id, Urls
# and additional information about their facial attributes.

display(getCelebrityRecognition)

In [ ]:
# Show video in a plyer

s3VideoUrl = "https://s3.amazonaws.com/{0}/{1}".format(bucketName, videoName)

videoTag = "<video controls='controls' autoplay width='800' height='600' name='Video' src='{0}'></video>".format(s3VideoUrl)

display(HTML(videoTag))

## Try recognizing custom celebrities 
***

In [ ]:
# Now let us try an image with non-celebtiries in the image.

customCelebrityImageName = "serverless-bytes.png"

In [ ]:
# Call Amazon Rekognition to recognize celebrities in the image

customCelebrityResponse = rekognition.recognize_celebrities(
    Image={
        'S3Object': {
            'Bucket': bucketName,
            'Name': customCelebrityImageName,
        }
    }
)

In [ ]:
# Display Rekognition response
# You will see Rekognition return and empty list for CelebrityFaces and 
# UnrecognizedFaces list with unrecognized faces that were detected in the image.
# In the next module you will learn how to get custom-celebrity faces recognized.

display(customCelebrityResponse)

In [ ]:
#Show image and bounded boxes around detected faces

# Extract BB info from response
cboxes = []
faces = customCelebrityResponse['UnrecognizedFaces']
for face in faces:
    cboxes.append (('Unrecognized Face', face['BoundingBox']))
    
drawBoundingBoxes(customCelebrityImageName, cboxes)

***
### References
- https://docs.aws.amazon.com/rekognition/latest/dg/celebrities.html
- https://docs.aws.amazon.com/rekognition/latest/dg/API_RecognizeCelebrities.html
- https://docs.aws.amazon.com/rekognition/latest/dg/API_StartCelebrityRecognition.html
- https://docs.aws.amazon.com/rekognition/latest/dg/API_GetCelebrityRecognition.html

***